In [2]:
import spotipy
import spotipy.util as util
import time
import timeit
import pandas as pd 
import numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
import json
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Preprocessing

In [3]:
# load data csv

df = pd.read_csv('df_by_playlist.csv')
print('dataframe has shape', df.shape)

dataframe has shape (1602, 63)


In [4]:
names = list(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence', 'explicit', 'mode', 'num_tracks',
       'track_feat', 'track_remast', 'track_remix', 'album_feat',
       'album_deluxe', 'album_best', 'time_sign_0.0', 'time_sign_3.0',
       'time_sign_4.0', 'key_0.0', 'key_1.0', 'key_2.0', 'key_3.0', 'key_4.0',
       'key_5.0', 'key_6.0', 'key_7.0', 'key_8.0', 'key_9.0', 'key_10.0','key_11.0'])

In [5]:
# normalize the (un-normalized) continuous data

colnames = ['duration_ms','popularity','tempo','num_tracks']
for name in colnames:
    df[name] = df[name]/max(df[name])

In [6]:
# add some interaction terms

quant_predictors = list([predictor for predictor in names if df.loc[:,predictor].unique().sum()!=1])

def interact(x,variable,predictors=quant_predictors):
    for predictor in predictors:
        x[variable+'_'+predictor] = x[variable]*x[predictor]
    return x

df = interact(df,'loudness')
df = interact(df,'danceability',list(['speechiness','tempo']))
df = interact(df,'loudness')
df = interact(df,'danceability',list(['speechiness','tempo']))

print('dataframe has shape', df.shape)

dataframe has shape (1602, 77)


In [7]:
# add polynomial features for continuous data

#colnames = ['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','popularity','speechiness','tempo','valence','num_tracks']
polydf = pd.DataFrame(df[quant_predictors])

deg = 2

poly = PolynomialFeatures(degree=deg)
polydftrans = poly.fit_transform(polydf)
target_feature_names = ['x'.join(['{}^{}'.format(pair[0],pair[1]) for pair in tuple if pair[1]!=0]) for tuple in [zip(polydf.columns,p) for p in poly.powers_]]
polydf = pd.DataFrame(polydftrans, columns = target_feature_names)

df = df.join(polydf)
df = df.drop(colnames,axis=1)
print('dataframe with continuous variable polynomial features has size', df.shape)

dataframe with continuous variable polynomial features has size (1602, 164)


In [8]:
# 50/50 train/test split

np.random.seed(9)
msk = np.random.rand(len(df)) < 0.5
dftr = df[msk]
dfte = df[~msk]

print('training set has shape', dftr.shape)
print('test set has shape', dfte.shape)

training set has shape (792, 164)
test set has shape (810, 164)


# Fit model

In [9]:
# do base line fit using only popularity

X_test = dfte['popularity^1']; X_test = sm.add_constant(X_test)
y_test = np.log(dfte['followers'])
X_train = dftr['popularity^1']; X_train = sm.add_constant(X_train)
y_train = np.log(dftr['followers'])

start = timeit.default_timer()
model = RidgeCV(alphas=(1e-5,1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3,1e4,1e5))
model.fit(X_train,y_train)

predtrain = model.predict(X_train)
predtest = model.predict(X_test)
stop = timeit.default_timer()

R2train = r2_score(y_train,predtrain)
R2test = r2_score(y_test,predtest)
print('Ridge alpha value is',model.alpha_)
print('R2 on training set is',R2train)
print('R2 on test set is',R2test)
print('took',stop - start,'seconds')

Ridge alpha value is 0.1
R2 on training set is 0.190246414835
R2 on test set is 0.186230526451
took 0.03254923224449158 seconds


In [10]:
# do fit using all predictors

X_test = dfte.drop(['playlist_name','followers'],axis=1); X_test = sm.add_constant(X_test)
y_test = np.log(dfte['followers'])
X_train = dftr.drop(['playlist_name','followers'],axis=1); X_train = sm.add_constant(X_train)
y_train = np.log(dftr['followers'])

start = timeit.default_timer()
model = RidgeCV(alphas=(1e-5,1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3,1e4,1e5))
model.fit(X_train,y_train)

predtrain = model.predict(X_train)
predtest = model.predict(X_test)
stop = timeit.default_timer()

R2train = r2_score(y_train,predtrain)
R2test = r2_score(y_test,predtest)
print('Ridge alpha value is',model.alpha_)
print('R2 on training set is',R2train)
print('R2 on test set is',R2test)
print('took',stop - start,'seconds')

Ridge alpha value is 0.1
R2 on training set is 0.578326119881
R2 on test set is 0.465611142266
took 0.019331875257194042 seconds
